<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-Garethlomax/blob/hdf5/HDF5_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDF5 CONVERSION FUNCTIONS
As our test datasets are too large to load into collab, we convert them to hdf5 format before using them. hdf5 is a standard data format for large scale datasets too big to be loaded nativelyt into ram. documentation can be found: 

http://docs.h5py.org/en/latest/quick.html#quick


# IMPORTS

%cd /content/drive/My \Drive/masters_project/python_modules/MovingMNIST-master

from MovingMNIST import MovingMNIST



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install h5py

import h5py
h5py.run_tests()



%cd /content/drive/My \Drive/masters_project/python_modules/MovingMNIST-master

from MovingMNIST import MovingMNIST



train_set = MovingMNIST(root='.data/mnist', train=True, download=False)
test_set = MovingMNIST(root='.data/mnist', train=False, download=False)



.....................................................x...................................................................x....................................s...s......ss.......................................................................................................ssssss...................................................................x....x.........................x......x.................................................ssss..................
----------------------------------------------------------------------
Ran 457 tests in 1.697s

OK (skipped=14, expected failures=6)


/content/drive/My Drive/masters_project/python_modules/MovingMNIST-master


# FUNCTION DEFINITIONS
Taken from mastersproject.ipynb 


In [0]:
def validation_split(data, n_splits = 1, valid_fraction = 0.1, random_state = 0):
    """
    Function to produce a validation set from test set.
    THIS SHUFFLES THE SAMPLES. __NOT__ THE SEQUENCES.
    """
    dummy_array = np.zeros(len(data))
    split = StratifiedShuffleSplit(n_splits, test_size = valid_fraction, random_state = 0)
    generator = split.split(torch.tensor(dummy_array), torch.tensor(dummy_array))
    return [(a,b) for a, b in generator][0]


In [0]:
def initialise_dataset(data):
    # unsqueeze data, adding a channel dimension for later convolution. 
    # this also gets rid of the annoying tuple format
    predictor, truth = unsqueeze_data(data)
    
    train_index, valid_index = validation_split(data)
    
    train_predictor = predictor[train_index]
    valid_predictor = predictor[valid_index]
    
    train_truth = truth[train_index]
    valid_truth = truth[valid_index]
    
    train_dataset = SequenceDataset(train_predictor, train_truth)
    valid_dataset = SequenceDataset(valid_predictor, valid_truth)
    
    return train_dataset, valid_dataset

In [0]:
def unsqueeze_data(data):
    """
    Takes in moving MNIST object - must then account for 
    """
    
    # split moving mnist data into predictor and ground truth.
    predictor = data[:][0].unsqueeze(2)
    predictor = predictor.double()
        
    truth = data[:][1].unsqueeze(2)# this should be the moving mnist sent in
    truth = truth.double()
    
    return predictor, truth

We now use unsqueeze data to produce data for saving into hdf5

we will have 2 datasets in each folder - prediction and truth. 






In [0]:
predictor, truth = unsqueeze_data(train_set)

# now convert to numpy
predictor = predictor.numpy()
truth = truth.numpy()


# FILE CREATION

In [7]:

%cd /content/drive/My \Drive/masters_project/data
# switch our file path to the data section. 


# create h5py
# to take advantage of their lazy loading.
f = h5py.File("train_set.hdf5", "w")
f.create_dataset("predictor", data = predictor)
f.create_dataset("truth", data = truth)
f.close()



/content/drive/My Drive/masters_project/data


In [9]:
!ls -F


mnist_test_seq.npy  test_set.hdf5  train_set.hdf5


# TESTING


In [0]:
f = h5py.File("train_set.hdf5", 'r')


In [14]:
f["predictor"]
# should be 9000 with 10 image sequence of 1 channel and 64 * 64 



<HDF5 dataset "predictor": shape (9000, 10, 1, 64, 64), type "<f8">

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],


        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],


        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [

In [8]:
list(f.keys())
f['predictor'][0]



ValueError: ignored

In [0]:
f.close()
